In [1]:
import pandas as pd

df_bluer = pd.read_csv('./data/bluer.csv')
df_michelin = pd.read_csv('./data/michelin.csv')

In [2]:
df_bluer = df_bluer.rename(columns={'review':'review_b'})
df_bluer = df_bluer.rename(columns={'priceRange':'priceRange_b'})
df_michelin = df_michelin.rename(columns={'review':'review_m'})
df_michelin = df_michelin.rename(columns={'priceRange':'priceRange_m'})

In [3]:
# remove whitespace (return, newline, tab)
for column in df_bluer.columns:
  df_bluer[column] = df_bluer[column].apply(str).str.replace(r'\r+|\n+|\t+',' ', regex=True)
for column in df_michelin.columns:
  df_michelin[column] = df_michelin[column].apply(str).str.replace(r'\r+|\n+|\t+',' ', regex=True)

In [4]:
def modify_phone_number(phone):
  if not (len(phone.split("-")) > 1):
    return
  parts = phone.split('-')
  if (len(parts[1]) == 5):
    swap = list(phone)
    swap[3], swap[4] = swap[4], swap[3]
    return ''.join(swap)
  return phone

In [5]:
df_michelin['phone'] = df_michelin['phone'].str.replace('+82 ', '0')
df_michelin['phone'] = df_michelin['phone'].str.replace('[]', '')
df_michelin['phone'] = df_michelin['phone'].apply(modify_phone_number)

In [6]:
df_merged = df_bluer.merge(df_michelin, on='phone', how='outer', suffixes=('_bluer', '_michelin'))

# Iterate over columns and prioritize df_bluer values, filling in from df_michelin where necessary
for column in df_bluer.columns:
    if column != 'phone':
        if column in df_michelin.columns:
            # Combine values from both DataFrames, prioritizing df_bluer
            df_merged[column] = df_merged[column + '_bluer'].combine_first(df_merged[column + '_michelin'])
            # Drop the temporary columns
            df_merged.drop(columns=[column + '_bluer', column + '_michelin'], inplace=True)
        else:
            # Just rename the column from df_bluer
            df_merged.rename(columns={column + '_bluer': column}, inplace=True)

# Handle columns that are only in df_michelin but not in df_bluer
for column in df_michelin.columns:
    if column not in df_bluer.columns and column != 'phone':
        # Rename the michelin columns to remove the suffix
        df_merged.rename(columns={column + '_michelin': column}, inplace=True)

df_merged = df_merged[['id', 'name', 'category', 'phone', 'address', 'latitude', 'longitude', 'businessHours', 'dayOff', 'parking', \
                       'services', 'menu', 'priceRange_b', 'priceRange_m', 'review_b', 'review_m', 'ribbonType', 'michelinType' ]]
df_merged = df_merged.sort_values(by='id')

# Export the final merged DataFrame to CSV
df_merged.to_csv('./data/processed_b_m.csv', index=False)

In [7]:
df_bluer.to_csv('./data/p_bluer.csv')
df_michelin.to_csv("./data/p_michelin.csv")